In [1]:
from urllib.request import urlopen


from bs4 import BeautifulSoup
import requests 
import lxml.html as lh
import re
import json
import datetime
import csv
import os

JsInfo={};
IndividualPlayer={};
AllPlayersLinks={};
AllPlayerInfo = {}
AllPlayerStats= {};
AllPlayerImg = {};
keys=["Player id", "Name", "Un1", "Year started", "Year ended", "Un2", "Un3"];
allPlayers=[];
playerIdList=[];
hrefLinkList=[];
hrefKeys=[];

url = "https://stats.nba.com/leaders/?Season=1951-52&SeasonType=Regular%20Season";
homeUrl="https://stats.nba.com";
html= urlopen(url);
bs = BeautifulSoup(html.read(), 'html.parser');

#Data From Js
for jslinks in bs.findAll('script', attrs={'src': re.compile("^/js/")}):
    jslink = homeUrl+jslinks.get('src');
bsJs = BeautifulSoup(urlopen(jslink).read(), 'html.parser');
jsdatapattern = re.compile(r"var stats_ptsd = (.*);", re.MULTILINE);
JsInfo = json.loads(re.search(jsdatapattern, str(bsJs.text)).group(1));
data1 = JsInfo['data'];
i=0;   
while i < len(data1["players"]):
    a=0;
    while a<len(data1["players"][i]):
        IndividualPlayer[keys[a]]=data1["players"][i][a];            
        a+=1;
    i+=1;
    allPlayers.append(dict(IndividualPlayer));#All Player details from Js
b=0;
while b<(len(allPlayers)):
    year = int(allPlayers[b]['Year started']);
    if(year==1950):
        playerIdList.append(allPlayers[b]['Player id']);#All Player ids who are from 1950 from Js
    b+=1;  

styleclass = bs.find('section',class_="dropdown__category");
for li in styleclass.find('li'):
    for link in styleclass.find_all('a'):
        if 'href' in link.attrs:
            hrefLinkList.append(homeUrl+link.attrs['href']);
            hrefKeys.append(link.text);
pp=0;
while pp<len(playerIdList):
    j=0;
    href={};
    while j<len(hrefLinkList):
        href[hrefKeys[j]]=hrefLinkList[j].replace('{{ player.id }}' ,str(playerIdList[pp]) );
        j+=1;
        AllPlayersLinks[playerIdList[pp]]=href;#All links for all Player ids who are from 1950 from website
    pp+=1;
 

p=0;
z={};
while(p< len(playerIdList)):
    url1 = AllPlayersLinks[playerIdList[p]]['Profile'];
    infopattern = re.compile(r"window\.nbaStatsPlayerInfo = (.*);", re.MULTILINE);
    statspattern = re.compile(r"window\.nbaStatsPlayerStats = (.*);", re.MULTILINE);
    response = requests.get(url1)
    soup = BeautifulSoup(response.content)   
    infoscript = soup.find("script", text=lambda x: x and "window.nbaStatsPlayerInfo" in x).text;
    statscript = soup.find("script", text=lambda x: x and "window.nbaStatsPlayerStats" in x).text;
    #All details for all Player ids who are from 1950 from website using their links
    AllPlayerInfo[playerIdList[p]]= json.loads(re.search(infopattern, infoscript).group(1)),json.loads(re.search(statspattern, statscript).group(1)), {'IMAGE' : soup.find('meta',property="og:image")["content"]};
    p+=1;
    

#print(AllPlayerInfo[playerIdList[0]])

keySet=[];
op=0;
while op<len(AllPlayerInfo[playerIdList[0]]):
    for key in AllPlayerInfo[playerIdList[0]][op].keys():
        keySet.append(key);
    op+=1;

#Prints details of each player individually in separate csv files with their id as filename
def IndividualFiles():
    pid=0;
    while pid < len(playerIdList):
        dir_name='WebScrapping/'
        base_filename=str(playerIdList[pid]);
        suffix='.csv'
        #print(os.path.join(dir_name, base_filename + suffix));
        with open(os.path.join(dir_name, base_filename + suffix), 'w') as f:
            i=0;a=0;
            while a < len(AllPlayerInfo[playerIdList[pid]]):
                b=0;
                while b < len(AllPlayerInfo[playerIdList[pid]][a]):
                    keyName = keySet[i];
                    f.write("%s,%s\n"%(keySet[i],AllPlayerInfo[playerIdList[pid]][a][keyName]));
                    i+=1;
                    b+=1;
                a+=1;
            pid+=1;

#Prints details of each player individually in single csv file
def SingleFile():
    pid=0;
    dir_name='WebScrapping/'
    base_filename='SingleFile';
    suffix='.csv'
    print(os.path.join(dir_name, base_filename + suffix));
    with open(os.path.join(dir_name, base_filename + suffix), 'w') as f:
         while pid < len(playerIdList):
            i=0;a=0;
            while a < len(AllPlayerInfo[playerIdList[pid]]):
                b=0;
                while b < len(AllPlayerInfo[playerIdList[pid]][a]):
                    keyName = keySet[i];
                    f.write("%s,%s\n"%(keySet[i],AllPlayerInfo[playerIdList[pid]][a][keyName]));
                    with open('data.json', 'w') as outfile:
                        json.dump((keySet[i],AllPlayerInfo[playerIdList[pid]][a][keyName]), outfile)
                    i+=1;
                    b+=1;
                a+=1;
            pid+=1;
    

#Prints details of each player individually in single Json file
def JsonFile():
    pid=0;
    with open('WebScrapping.json', 'w') as outfile:
         while pid < len(playerIdList):
            i=0;a=0;
            while a < len(AllPlayerInfo[playerIdList[pid]]):
                b=0;
                while b < len(AllPlayerInfo[playerIdList[pid]][a]):
                    keyName = keySet[i];
                    json.dump((keySet[i],AllPlayerInfo[playerIdList[pid]][a][keyName]), outfile)
                    i+=1;
                    b+=1;
                a+=1;
            pid+=1;
    
#SingleFile();
#IndividualFiles();
JsonFile();

({'PERSON_ID': 76056, 'FIRST_NAME': 'Paul', 'LAST_NAME': 'Arizin', 'DISPLAY_FIRST_LAST': 'Paul Arizin', 'DISPLAY_LAST_COMMA_FIRST': 'Arizin, Paul', 'DISPLAY_FI_LAST': 'P. Arizin', 'BIRTHDATE': '1928-04-09T00:00:00', 'SCHOOL': 'Villanova', 'COUNTRY': 'USA', 'LAST_AFFILIATION': 'Villanova/USA', 'HEIGHT': '6-4', 'WEIGHT': '190', 'SEASON_EXP': 9, 'JERSEY': '11', 'POSITION': 'Forward', 'ROSTERSTATUS': 'Active', 'TEAM_ID': 1610612744, 'TEAM_NAME': 'Warriors', 'TEAM_ABBREVIATION': 'PHW', 'TEAM_CODE': 'warriors', 'TEAM_CITY': 'Philadelphia', 'PLAYERCODE': 'HISTADD_paul_arizin', 'FROM_YEAR': 1950, 'TO_YEAR': 1961, 'DLEAGUE_FLAG': 'N', 'NBA_FLAG': 'Y', 'GAMES_PLAYED_FLAG': 'Y', 'DRAFT_YEAR': '1950', 'DRAFT_ROUND': None, 'DRAFT_NUMBER': None, 'POSITION_INITIALS': 'F', 'AGE': 90, 'AGE_DAYS': '313'}, {'PLAYER_ID': 76056, 'PLAYER_NAME': 'Paul Arizin', 'TimeFrame': 'career', 'PTS': 22.8, 'AST': 2.3, 'REB': 8.6, 'ALL_STAR_APPEARANCES': 9}, {'IMAGE': 'https://ak-static.cms.nba.com/wp-content/uploads/he